# Mushroom Classification Project (15 Epoch Optimized Version)
This notebook contains two models:
1. **Model 1**: CNN classifier with Softmax (optimized for 15 epochs)
2. **Model 2**: CNN feature extractor + Linear SVM

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


In [2]:
# Load dataset
data_dir = 'Mushrooms'
classes = ['Agaricus', 'Amanita', 'Boletus', 'Cortinarius','Entoloma','Hygrocybe','Lactarius', 'Russula', 'Suillus']
img_height, img_width = 224, 224

images, labels = [], []
for label in classes:
    folder = os.path.join(data_dir, label)
    for img_file in os.listdir(folder):
        img_path = os.path.join(folder, img_file)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (img_width, img_height))
            images.append(img)
            labels.append(label)

images = np.array(images, dtype=np.float32)
mean = np.mean(images, axis=(0, 1, 2))
std = np.std(images, axis=(0, 1, 2))
images = (images - mean) / std

le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
labels_cat = to_categorical(labels_encoded)


In [3]:
x_train, x_val, y_train, y_val = train_test_split(images, labels_cat, test_size=0.2, random_state=42)

datagen = ImageDataGenerator(
    rotation_range=12,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)


In [ ]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import LeakyReLU

input_layer = Input(shape=(img_height, img_width, 3))

x = Conv2D(32, (3, 3), kernel_initializer='he_normal', kernel_regularizer=l2(1e-4), padding='same')(input_layer)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = MaxPooling2D()(x)

x = Conv2D(64, (3, 3), kernel_initializer='he_normal', kernel_regularizer=l2(1e-4), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = MaxPooling2D()(x)

x = Conv2D(128, (3, 3), kernel_initializer='he_normal', kernel_regularizer=l2(1e-4), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = MaxPooling2D()(x)

x = Flatten()(x)
x = Dense(128, kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(x)
x = LeakyReLU()(x)
x = Dropout(0.3)(x)
output_layer = Dense(len(classes), activation='softmax')(x)

model1 = Model(inputs=input_layer, outputs=output_layer)
optimizer = Adam(learning_rate=0.0002)
model1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy()])
model1.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    12,845,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,940,489 (49.36 MB)

 Trainable params: 12,940,041 (49.36 MB)

 Non-trainable params: 448 (1.75 KB)

: 

In [ ]:
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]

history = model1.fit(datagen.flow(x_train, y_train, batch_size=32),
                     validation_data=(x_val, y_val),
                     epochs=50,
                     callbacks=callbacks)

model1.save("model1_softmax.h5")


C:\Users\Dhruv\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
168/168 ━━━━━━━━━━━━━━━━━━━━ 340s 2s/step - categorical_accuracy: 0.2185 - loss: 11.0596 - val_categorical_accuracy: 0.3142 - val_loss: 5.5973 - learning_rate: 2.0000e-04
Epoch 2/50
168/168 ━━━━━━━━━━━━━━━━━━━━ 324s 2s/step - categorical_accuracy: 0.2741 - loss: 5.8548 - val_categorical_accuracy: 0.3090 - val_loss: 5.6867 - learning_rate: 2.0000e-04
Epoch 3/50
168/168 ━━━━━━━━━━━━━━━━━━━━ 325s 2s/step - categorical_accuracy: 0.2971 - loss: 4.7930 - val_categorical_accuracy: 0.3098 - val_loss: 4.4970 - learning_rate: 2.0000e-04
Epoch 4/50
168/168 ━━━━━━━━━━━━━━━━━━━━ 323s 2s/step - categorical_accuracy: 0.3052 - loss: 4.3642 - val_categorical_accuracy: 0.3470 - val_loss: 4.1252 - learning_rate: 2.0000e-04
Epoch 5/50
168/168 ━━━━━━━━━━━━━━━━━━━━ 321s 2s/step - categorical_accuracy: 0.3126 - loss: 3.8620 - val_categorical_accuracy: 0.3485 - val_loss: 3.1724 - learning_rate: 2.0000e-04
Epoch 6/50
168/168 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - categorical_accuracy: 0.3027 - loss: 3.

In [ ]:
feature_extractor = Model(inputs=model1.input, outputs=model1.get_layer(index=-3).output)

x_train_feat = feature_extractor.predict(x_train)
x_val_feat = feature_extractor.predict(x_val)

y_train_svm = np.argmax(y_train, axis=1)
y_val_svm = np.argmax(y_val, axis=1)

svm_model = SVC(kernel='linear')
svm_model.fit(x_train_feat, y_train_svm)

y_pred = svm_model.predict(x_val_feat)
acc = accuracy_score(y_val_svm, y_pred)
print(f"SVM Model Accuracy: {acc:.4f}")

joblib.dump(best_svm, "svm_model.joblib")
print("Model 2 (SVM) saved as svm_model.joblib")
